In [ ]:
import os.path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
def plot_auc(paths, sizes):
    auc_mean = []
    auc_stderr = []
    auc_all = []
    for path in paths:
        auc = np.load(path)
        # auc = auc[auc>auc.min()]
        # auc = auc[auc<auc.max()]
        auc_mean.append(np.median(auc))
        auc_stderr.append(1.96*np.std(auc)/np.sqrt(len(auc)))
        auc_all.append(auc)
    
    plt.figure()
    plt.errorbar(sizes, auc_mean, auc_stderr, fmt='.')
    return auc_mean, auc_stderr, auc_all

In [ ]:
labels = np.load("data/labels_ecg.npy")

In [ ]:
len(labels)

In [ ]:
labels.sum()

In [ ]:
5023+8354

In [ ]:
labels.sum() / len(labels)

In [ ]:
res = pd.read_csv("results/splitting_method_3/sample_10000/AFprediction_sample10000_250Hz_50reps_correct.csv")
res = res.drop(columns=["config"])

In [ ]:
sizes = [3500, 4500, 5500, 6500, 7500]
path = "results/splitting_method_3/sample_10000/" # method 3, sample 2000, 100 repetitions, stable config
paths = [path + "/" + str(size) + "_2024-11-09/auc_test.npy" for size in sizes]
auc_mean, auc_stderr, auc_all = plot_auc(paths, sizes)
res.columns = res.columns.values.astype(int)

In [ ]:
auc_all

In [ ]:
for i, size in enumerate(sizes):
    res[size] = auc_all[i]

In [ ]:
plt.plot(res.columns, res.mean(), '.')

In [ ]:
res = res[np.sort(res.columns)]

In [ ]:
res
plt.plot(res.columns, res.mean(), '.')

In [ ]:
res.to_csv("results/splitting_method_3/sample_10000/AFprediction_sample10000_250Hz_50reps.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.reset_defaults()
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("results/splitting_method_3/sample_10000/AFprediction_sample10000_250Hz_50reps.csv")

In [ ]:
n = df.columns.values.astype(int)
std_err = 1.96 *df.std() / np.sqrt(df.shape[0])

In [ ]:
n_weights = 1 / n

In [ ]:
sns.histplot(df["8000"], bins=10)

In [ ]:
plt.plot(std_err, '.-', label="Standard error")
plt.plot(n_weights, '.-', label="1/N")
plt.xticks(rotation=45)
plt.legend()
plt.yscale("log")
plt.ylim((0.0001, 0.02))
plt.savefig("results/stdErr_vs_cardinality.jpg", dpi=300)
plt.show()

In [ ]:
plt.plot(std_err / n_weights, ".-",
        label="(Standard error) / (1/N)", linewidth=0.5)
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
# df = pd.read_csv("results/AFdetection_sample1000_100reps.csv")
# n = df.columns.values.astype(int)
# std_err = df.std() / np.sqrt(df.shape[0])
# plt.plot(std_err, '.-', label="Standard error")
# plt.plot(1/n, '.-', label="1/N")
# plt.xticks(rotation=45)
# plt.legend()
# plt.yscale("log")
# plt.ylim((0.001, 0.0105))
# plt.show()

In [ ]:
def sigma(x, threshold):
    r = (x - threshold) / 1000
    return 1 / (1+ np.exp(-r))

In [ ]:
def f_exp(x, a, b, c):
    return  a * np.exp(-b*x) + c

# def f_pow3_exp(x, a, b, e, c):
#     return sigma(x, 5000) * a * x**(-b) + sigma(x, 5000) * a * np.exp(-e*x) + c

def f(x, a, b, c):
    return a * x**(-b) + c

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
x = df.columns.astype(int)
y = df.median().values
std_err = 1.96 * df.std() / np.sqrt(df.shape[0])

In [ ]:
popt_exp, pcov_exp  = curve_fit(f=f_exp, xdata=x, ydata=y, sigma=std_err,
                                p0=(-1, 0, 0.83),
                                bounds=((-10, 0, 0.7),(0, 100, 0.9999)),
                                maxfev=10000)
plt.errorbar(x=x, y=y, yerr=std_err, fmt=".")
plt.plot(np.linspace(x.min(), x.max(), 100), f_exp(np.linspace(x.min(), x.max(), 100), *popt_exp))
plt.show()

In [ ]:
def f_pow3_exp(x, a, b, c, e):
    return sigma(x, 5000) * a * x**(-b) + sigma(x, 5000) * a * np.exp(-e*x) + c

def smooth_piecewise(x, x_threshold, k, a1, b1, c1, a2, b2, c2):
    """
    Smooth piecewise function transitioning between:
    f1(x) = a1 * x^(-b1) + c1 (for x < x_threshold)
    f2(x) = a2 * exp(-b2 * x) + c2 (for x >= x_threshold)
    
    Parameters:
        x: Independent variable (array-like).
        x_threshold: Threshold where the transition happens.
        k: Smoothness/sharpness of the transition (higher k = sharper).
        a1, b1, c1: Parameters for f1(x).
        a2, b2, c2: Parameters for f2(x).
    
    Returns:
        Smoothly transitioned values of f(x).
    """
    # Sigmoid function for smooth transition
    sigmoid = 1 / (1 + np.exp(-k * (x - x_threshold)))
    
    # Define the two behaviors
    f1 = a1 * x**(-b1) + c1  # For x < x_threshold
    f2 = a2 * np.exp(-b2 * x) + c2  # For x >= x_threshold
    
    # Smooth transition
    return (1 - sigmoid) * f1 + sigmoid * f2

popt_pow3_exp, pcov_pow3_exp  = curve_fit(f=f_pow3_exp, xdata=x, ydata=y, sigma=std_err,
                                          p0=(-3, 0.4, 0.9, 0),
                                          bounds=((-4, 0, 0, 0),(0, 1, 0.9999, 1)),
                                          maxfev=10000)
plt.errorbar(x=x, y=y, yerr=std_err, fmt=".")
plt.plot(np.linspace(x.min(), x.max(), 100), f_pow3_exp(np.linspace(x.min(), x.max(), 100), *popt_pow3_exp))
chi2 = np.sum(((y - f_pow3_exp(x, *popt_pow3_exp)) / std_err) ** 2)
dof = len(y) - len(popt_pow3_exp)  # Number of data points - number of parameters
reduced_chi2 = chi2 / dof
plt.text(4000, 0.68, f"$\chi^2$ ridotto = {np.round(reduced_chi2, 3)}")
plt.text(4000, 0.672, f"a = {np.round(popt_pow3_exp[0], 3)} +/- {np.round(np.sqrt(np.diag(pcov_pow3_exp)[0]), 3)}")
plt.text(4000, 0.664, f"b = {np.round(popt_pow3_exp[1], 3)} +/- {np.round(np.sqrt(np.diag(pcov_pow3_exp)[1]), 3)}")
plt.text(4000, 0.656, f"c = {np.round(popt_pow3_exp[2], 3)} +/- {np.round(np.sqrt(np.diag(pcov_pow3_exp)[2]), 3)}")
plt.text(4000, 0.648, f"e = {np.round(popt_pow3_exp[3], 3)} +/- {np.round(np.sqrt(np.diag(pcov_pow3_exp)[3]), 3)}")
plt.show()

In [ ]:
popt_pow3_exp

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12, 5))
std_err = 1.96 *df.std() / np.sqrt(df.shape[0])
popt1, pcov1 = curve_fit(f=f, xdata=x, ydata=y, sigma=std_err, p0=(-1, 0, 0.83), bounds=((-100, 0, 0.7), (0, 100, 0.99)), maxfev=10000)
ax1.errorbar(x=x, y=y, yerr=std_err, fmt=".")
ax1.plot(np.linspace(x.min(), x.max(), 100), f(np.linspace(x.min(), x.max(), 100), *popt1))

# Reduced chi-squared
chi2 = np.sum(((y - f(x, *popt1)) / std_err)**2)
dof = len(y) - len(popt1)  # Number of data points - number of parameters
reduced_chi2 = chi2 / dof
ax1.set_title("Errore standard")
ax1.text(4000, 0.7, r"$f(x) = ax^{-b} + c$")
ax1.text(4000, 0.68, f"$\chi^2$ ridotto = {np.round(reduced_chi2, 3)}")
ax1.text(4000, 0.672, f"a = {np.round(popt1[0], 3)} +/- {np.round(np.sqrt(np.diag(pcov1)[0]), 3)}")
ax1.text(4000, 0.664, f"b = {np.round(popt1[1], 3)} +/- {np.round(np.sqrt(np.diag(pcov1)[1]), 3)}")
ax1.text(4000, 0.656, f"c = {np.round(popt1[2], 3)} +/- {np.round(np.sqrt(np.diag(pcov1)[2]), 3)}")

err_card = 1 / x
popt2, pcov2 = curve_fit(f=f, xdata=x, ydata=y, sigma=err_card, p0=(-1, 0, 0.83),
                         bounds=((-100, 0, 0.7), (0, 100, 0.99)), maxfev=10000)
ax2.errorbar(x=x, y=y, yerr=err_card, fmt=".")
ax2.plot(np.linspace(x.min(), x.max(), 100), f(np.linspace(x.min(), x.max(), 100), *popt2))

# Reduced chi-squared
chi2 = np.sum(((y - f(x, *popt2)) / err_card) ** 2)
dof = len(y) - len(popt2)  # Number of data points - number of parameters
reduced_chi2 = chi2 / dof
ax2.set_title("1/N")
ax2.text(4000, 0.7, r"$f(x) = ax^{-b} + c$")
ax2.text(4000, 0.682, f"$\chi^2$ ridotto = {np.round(reduced_chi2, 3)}")
ax2.text(4000, 0.675, f"a = {np.round(popt2[0], 3)} +/- {np.round(np.sqrt(np.diag(pcov2)[0]), 3)}")
ax2.text(4000, 0.668, f"b = {np.round(popt2[1], 3)} +/- {np.round(np.sqrt(np.diag(pcov2)[1]), 3)}")
ax2.text(4000, 0.661, f"c = {np.round(popt2[2], 3)} +/- {np.round(np.sqrt(np.diag(pcov2)[2]), 3)}")
plt.savefig("results/fit_both.png", dpi=300)
plt.show()

In [ ]:
plt.errorbar(x=x, y=y, yerr=std_err, fmt=".", label="data")
x_dense = np.linspace(x.min(), x.max(), 100) 
plt.plot(x_dense, f(x_dense, *popt1), label="Standard Error")
plt.plot(x_dense, f(x_dense, *popt1), label="1/N")
plt.legend()
plt.show()

In [ ]:
df_test = pd.read_csv("results/AFprediction_sample224000_250Hz_5reps.csv")

plt.figure()
plt.errorbar(x=x, y=y, yerr=std_err, fmt=".", label="training data")
plt.errorbar(x=df_test.columns.astype(int), y=df_test.mean(), yerr=1.96*df_test.std()/np.sqrt(df_test.shape[0]), fmt=".", label="test data")
x_dense = np.linspace(x.min(), 200000, 100) 
plt.plot(x_dense, f(x_dense, *popt1), label="Standard Error")
plt.plot(x_dense, f(x_dense, *popt2), label="1/N")
plt.plot(x_dense, f_exp(x_dense, *popt_exp), label="exp")
plt.plot(x_dense, f_pow3_exp(x_dense, *popt_pow3_exp), label="pow2+exp")
plt.legend()
plt.savefig("results/fit_test_both.png", dpi=300)
plt.show()

In [ ]:
# Chi-Square Goodness of Fit Test
chi2 = np.sum(((y - f1(x, *popt1)) / std_err)**2)

# Reduced chi-squared
dof = len(y) - len(popt1)  # Number of data points - number of parameters
reduced_chi2 = chi2 / dof
reduced_chi2

In [ ]:
popt1

In [ ]:
# Chi-Square Goodness of Fit Test
chi2 = np.sum(((y - f(x, *popt2)) / std_err)**2)

# Reduced chi-squared
dof = len(y) - len(popt2)  # Number of data points - number of parameters
reduced_chi2 = chi2 / dof
reduced_chi2

In [ ]:
popt2

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Define the smooth piecewise function
def smooth_piecewise(x, x_threshold, k, a1, b1, c1, a2, b2, c2):
    """
    Smooth piecewise function transitioning between:
    f1(x) = a1 * x^(-b1) + c1 (for x < x_threshold)
    f2(x) = a2 * exp(-b2 * x) + c2 (for x >= x_threshold)
    
    Parameters:
        x: Independent variable (array-like).
        x_threshold: Threshold where the transition happens.
        k: Smoothness/sharpness of the transition (higher k = sharper).
        a1, b1, c1: Parameters for f1(x).
        a2, b2, c2: Parameters for f2(x).
    
    Returns:
        Smoothly transitioned values of f(x).
    """
    # Sigmoid function for smooth transition
    sigmoid = 1 / (1 + np.exp(-k * (x - x_threshold)))
    
    # Define the two behaviors
    f1 = a1 * x**(-b1) + c1  # For x < x_threshold
    f2 = a2 * np.exp(-b2 * x) + c2  # For x >= x_threshold
    
    # Smooth transition
    return (1 - sigmoid) * f1 + sigmoid * f2

# Generate synthetic data for testing
np.random.seed(42)
x_data = np.linspace(0.5, 10, 100)  # Avoid x=0 due to division
y_data = np.piecewise(
    x_data,
    [x_data < 5, x_data >= 5],
    [
        lambda x: -0.3 * x**(-1.5) + 0.9,  # a1=2, b1=1.5, c1=0.9
        lambda x: -0.3 * np.exp(-0.5 * x) + 0.9,  # a2=5, b2=0.5, c2=0.9
    ],
) + np.random.normal(0, 0.01, len(x_data))  # Add noise

# Initial guesses for parameters
initial_guess = [5, 10, 2, 1.5, 3, 5, 0.5, 2]

# Fit the smooth piecewise function to the data
params, covariance = curve_fit(smooth_piecewise, x_data, y_data, p0=initial_guess, maxfev=1000000)

# Plot the results
plt.scatter(x_data, y_data, label="Data", color="blue", s=10)
plt.plot(
    x_data,
    smooth_piecewise(x_data, *params),
    label="Fitted Curve",
    color="red",
    linewidth=2,
)
plt.axvline(params[0], color="green", linestyle="--", label="Threshold (x_threshold)")
plt.legend()
plt.xlabel("x")
plt.ylabel("y")
plt.title("Smooth Piecewise Fit")
plt.show()

# Print fitted parameters
print("Fitted Parameters:")
print(f"x_threshold = {params[0]:.3f}, k = {params[1]:.3f}")
print(f"a1 = {params[2]:.3f}, b1 = {params[3]:.3f}, c1 = {params[4]:.3f}")
print(f"a2 = {params[5]:.3f}, b2 = {params[6]:.3f}, c2 = {params[7]:.3f}")
